In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
from src.settings import *
from src.tools.osmnx_utils import get_place_dir_name
import json5 as json
from functional import seq

tqdm.pandas()

In [3]:
with open(RAW_DATA_DIR / "featureset_transformation_default.jsonc", "r") as f:
    featureset = json.load(f)
    features = list(featureset.keys())

In [19]:
cities = pd.read_csv(RAW_DATA_DIR / "cities.csv")
# cities = cities[(cities.kacper)]
# cities = cities[(cities["country"] == "Poland")]
# cities = cities[(cities["city"] == "Wrocław")]

cities

,city,country,continent,kacper,szymon,piotr,kamil,regions,to_fix
85,Wrocław,Poland,Europe,True,True,True,True,NaN,False


In [20]:
network_type = "drive"

pbar = tqdm(cities.itertuples(), total=cities.shape[0])
edges_cities = []
for row in pbar:
    place_name = f"{row.city},{row.country}"
    place_dir_name = get_place_dir_name(place_name)
    pbar.set_description(place_name)

    try:
        edges_city = gpd.read_file(GENERATED_DATA_DIR / place_dir_name / f"graph_{network_type}.gpkg", layer="edges")
        edges_city["city"] = row.city
        edges_city["country"] = row.country
        edges_city["continent"] = row.continent
        edges_cities.append(edges_city)
    except Exception as e:
        print("\nFailed", place_name, e)
    
columns_superset = seq(edges_cities).map(lambda edges_city: set(edges_city.columns)).reduce(lambda a, b: a.union(b))
columns_superset

for edges_city in edges_cities:
    edges_city_cols = set(edges_city.columns)
    missing_columns = list(columns_superset - edges_city_cols)
    edges_city[missing_columns] = None

edges = pd.concat(edges_cities, ignore_index=True)
del edges_city, edges_cities

Wrocław,Poland: 100%|██████████| 1/1 [00:00<00:00,  1.84it/s]


In [21]:
edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9732 entries, 0 to 9731
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   u          9732 non-null   int64   
 1   v          9732 non-null   int64   
 2   key        9732 non-null   int64   
 3   osmid      9732 non-null   object  
 4   oneway     9732 non-null   bool    
 5   lanes      9732 non-null   object  
 6   name       9732 non-null   object  
 7   highway    9732 non-null   object  
 8   maxspeed   9732 non-null   object  
 9   surface    9732 non-null   object  
 10  lit        9732 non-null   object  
 11  length     9732 non-null   float64 
 12  from       9732 non-null   int64   
 13  to         9732 non-null   int64   
 14  ref        9732 non-null   object  
 15  bridge     9732 non-null   object  
 16  access     9732 non-null   object  
 17  bicycle    9732 non-null   object  
 18  junction   9732 non-null   object  
 19  width      9732 non

In [22]:
edges.columns

Index(['u', 'v', 'key', 'osmid', 'oneway', 'lanes', 'name', 'highway',
       'maxspeed', 'surface', 'lit', 'length', 'from', 'to', 'ref', 'bridge',
       'access', 'bicycle', 'junction', 'width', 'tunnel', 'geometry', 'city',
       'country', 'continent'],
      dtype='object')

In [24]:
from itertools import chain
from src.tools.feature_extraction import sanitize, normalize, sanitize_and_normalize
from IPython.display import clear_output
cols_bloat = set(['u', 'v', 'geometry', 'osmid', 'from', 'to', 'ref', 'name', 'key', 'city', 'country', 'continent'])

path = RAW_DATA_DIR / "features_counts"
path.mkdir(exist_ok=True, parents=True)

pbar = tqdm((set(columns_superset) - cols_bloat))
for f in pbar:
    pbar.set_description(f)

    edges_list = edges[f].apply(lambda x: eval(str(x)) if "[" in str(x) else [str(x)])
    sanitized = edges_list.apply(lambda x: [sanitize(y, f) for y in x])
    normalized = sanitized.apply(lambda x: [normalize(y, f) for y in x])

    counts = edges_list.explode().value_counts().sort_values(ascending=False)
    counts_sanitized = sanitized.explode().value_counts().sort_values(ascending=False)
    counts_normalized = normalized.explode().value_counts().sort_values(ascending=False)

    clear_output(wait=True)

    # edges_list.to_csv(path / f"{f}.csv")
    # sanitized.to_csv(path / f"{f}_sanitized.csv")
    # normalized.to_csv(path / f"{f}_normalized.csv")

    counts.to_csv(path / f"{f}_counts.csv")
    counts_sanitized.to_csv(path / f"{f}_sanitized_counts.csv")
    counts_normalized.to_csv(path / f"{f}_normalized_counts.csv")
    
    

bridge: 100%|██████████| 13/13 [00:01<00:00, 11.59it/s]
